In [ ]:
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from datetime import date, timedelta
from selenium import webdriver
import requests
import time
import os

In [ ]:
# Initialize the WebDriver (Here, we use Chrome)
driver = webdriver.Chrome()

# Step 2: Navigate to the webpage and download the file
url = "https://dps.psx.com.pk/downloads"  # Replace with the target URL
driver.get(url)

# Start and end dates
start_date = date(2025, 6, 18)
end_date = date(2025, 10, 1)

# Loop through the dates
current_date = start_date
while current_date <= end_date:
    if current_date.weekday() not in (5, 6):
        # print(current_date)
        # Step 3: Input a custom date (if the webpage supports it)
        date_field_selector = "input#downloadsDatePicker"  # Replace with the actual selector for the date input field
        custom_date = current_date.strftime("%Y-%m-%d")  # Replace with the desired date (format YYYY-MM-DD)

        # Find the date input field and set its value
        date_field = driver.find_element(By.CSS_SELECTOR, date_field_selector)
        date_field.clear()  # Clear any existing value
        date_field.send_keys(custom_date)  # Input the custom date

        # If the page requires submitting the date, you might also need to find and click the submit button
        submit_button_selector = "button#downloadsSearchBtn"  # Replace with the actual selector
        submit_button = driver.find_element(By.CSS_SELECTOR, submit_button_selector)
        submit_button.click()

        # Pause for the page to update after entering the custom date
        time.sleep(5)  # Adjust the sleep duration as needed

        try:
            # Attempt to locate the element
            download_link = driver.find_element(By.CSS_SELECTOR, "div.downloads > div:nth-last-child(2) a.xls").get_attribute("href")

            # Step 4: Use requests to download the file
            if download_link:
                base_url = "https://dps.psx.com.pk/downloads"  # Replace with the actual base URL of the site if needed
                full_url = download_link if download_link.startswith("http") else base_url + download_link
                print(f"Downloading file from: {full_url}")

                # Extract the file name from the URL
                file_name = os.path.basename(download_link)  # Gets the last part of the URL (e.g., "2024-11-28.xls")
                print(f"Original file name: {file_name}")
                
                # Save the file locally
                download_dir = "E:\\Codes\\Python\\indexfundproject\\dataextract\\data"  # Replace with your desired download path
                local_file_path = os.path.join(download_dir, file_name)
                
                # Download the file
                response = requests.get(full_url, allow_redirects=True)
                with open(local_file_path, "wb") as file:
                    file.write(response.content)
                
                print(f"File downloaded to: {local_file_path}")
            else:
                print("Download link not found!")  # Replace with the actual selector
            # Wait for the file to download
            time.sleep(5)  # Adjust based on file size and download speed
        except NoSuchElementException:
                # Handle the case where the element does not exist
                print("Download link does not exist.")

    current_date += timedelta(days=1)

# Clean up and close the browser
driver.quit()